<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/CNN_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd 
import sklearn
import gc


# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import metrics
from sklearn.metrics import classification_report

# Prepare Data

In [1]:
def makedata(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    # X = np.array(X)
    return X

def scale_down(X):
  if(X.shape[0]<70000):
    return X
  X_new = []
  for i in range(int(X.shape[0]/2)):
    cur_row = X[i*2]
    for j in range(1,2):
      if(i+j < X.shape[0]):
        cur_row += X[i+j]
    cur_row = cur_row/2.0
    X_new.append(cur_row)
  X_new = np.array(X_new)
  return X_new

def decrease_size(X):
  size = 30000
  if(X.shape[0]<size):
    dif = size - X.shape[0] 
    temp = np.zeros((dif,X.shape[1]))
    X = np.concatenate((X,temp),axis = 0)
  if(X.shape[0]>size):
    X = X[:30000]
  return X

def Thresholding(Y_pred, threshold):
  Y_pred2 = []
  print("Y_pred: ", Y_pred.shape)
  for i in range(len(Y_pred)):
    if(Y_pred[i] < threshold):
      Y_pred2.append(0)
    else:
      Y_pred2.append(1)

  return np.array(Y_pred2)

def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train==0]
  X_train_1 = X_train[Y_train==1]

  Y_train_1 = Y_train[Y_train==1]
  # print(Y_train_1.shape)
  # print(X_train_1.shape)
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train


dev = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
test = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
train = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

X_train = []
Y_train = []

X_test = []
Y_test = []
size = 40000

for i in range(len(test)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/test_data/'+str(int(test[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_test.append(X_temp)
    Y_test.append(test[i][1])

X_test = np.array(X_test)
Y_test = np.array(Y_test)      
test = []
gc.collect()

for i in range(len(train)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/train_data/'+str(int(train[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_train.append(X_temp)
    Y_train.append(train[i][1])

train = []
gc.collect()

for i in range(len(dev)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_data/'+str(int(dev[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_train.append(X_temp)
    Y_train.append(dev[i][1])

X_temp = []
dev = []
gc.collect()

X_train = np.array(X_train, dtype = object)
Y_train = np.array(Y_train)

X_upsample, Y_upsample = upsample(X_train,Y_train)
# print(X_upsample.shape)
X_train = []
Y_train = []
gc.collect()

NameError: name 'np' is not defined

# CNN Model For Audio

In [ ]:
class CNN_audio:

  def __init__(self):
# Initialising the CNN
    classifier = Sequential()
# Step 1 - Convolution
    classifier.add(Conv1D(60, 10, input_shape = (40000, 74), activation = 'relu'))
# Step 2 - Pooling
    classifier.add(MaxPooling1D(pool_size = 3))
# Adding a second convolutional layer
    classifier.add(Conv1D(30, 5, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 3))
    classifier.add(Conv1D(15, 5, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 3))
# Step 3 - Flattening
    classifier.add(Flatten())
    classifier.add(Dropout(0.5))
# Step 4 - Full connection
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))
# classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    self.classifier = classifier
  
  def modelFit(self, X, Y, epoch = 10):
    self.classifier.fit(X, Y, epochs=epoch)

  def modelPredict(self, X):
    return self.classifier.predict(X)

model = CNN_audio()
model.modelFit(X_upsample, Y_upsample, 1)
Y_pred = Thresholding(model.modelPredict(X_test), 0.8)
print(classification_report(Y_test,Y_pred))

Epoch 1/1
196/196 [==============================] - 17s 86ms/step - loss: 88.4277 - accuracy: 0.5153
Y_pred:  (41, 1)
[1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0
 1 1 0 0]
              precision    recall  f1-score   support

           0       0.83      0.34      0.49        29
           1       0.34      0.83      0.49        12

    accuracy                           0.49        41
   macro avg       0.59      0.59      0.49        41
weighted avg       0.69      0.49      0.49        41



In [ ]:
# classifier = Sequential()

# # Step 1 - Convolution
# classifier.add(Conv1D(32, 3, input_shape = (6000, 74), activation = 'relu'))

# # Step 2 - Pooling
# classifier.add(MaxPooling1D(pool_size = 2))

# # Adding a second convolutional layer
# classifier.add(Conv1D(32, 3, activation = 'relu'))
# classifier.add(MaxPooling1D(pool_size = 2))

# # Step 3 - Flattening
# classifier.add(Flatten())

# # Step 4 - Full connection
# classifier.add(Dense(units = 128, activation = 'relu'))
# classifier.add(Dense(units = 1, activation = 'sigmoid'))

# # Compiling the CNN
# classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



# classifier.fit(np.array([X_train]), Y_train, epochs=2)
# score = classifier.evaluate([X_test], Y_test)


